# Análisis de datos transcriptómicos

## Pregunta

> ¿Cómo podemos identificar genes diferencialmente expresados entre dos condiciones biológicas a partir de datos de RNA-Seq?

## Objetivos

**General:**

> Brindar una visión general del flujo de trabajo para el **análisis de expresión diferencial** a partir de datos de RNA-Seq, que permita comparar dos **condiciones experimentales** e identificar los genes con **diferencias de expresión estadísticamente significativas**, utilizando **Python** y la biblioteca **pydeseq2**.

**Específicos:**

> 1. Comprender las etapas principales del flujo de análisis de expresión diferencial (normalización, estimación de dispersión, pruebas estadísticas).
> 2. Utilizar el paquete pyDESeq para realizar un análisis de expresión diferencial a partir de una matriz de conteos y metadatos experimentales.
> 3. Interpretar los resultados del análisis, incluyendo valores de fold change, p-values y adjusted p-values.
> 4.Generar visualizaciones básicas (como gráficos MA o volcanos) para resumir los resultados y facilitar su interpretación.
> 5. Consolidar el flujo completo desde la carga de datos hasta la obtención de resultados biológicamente interpretables.

## 1. ¿Qué es el análisis de expresión diferencial?

Un organismo puede tener miles de genes; algunos de ellos están siempre activos y son necesarios para su funcionamiento básico. Sin embargo, existen otros genes cuya expresión puede “apagarse” o “encenderse” en respuesta a determinadas condiciones ambientales o funcionales.

Los estudios de **expresión diferencial** buscan precisamente determinar **qué genes cambian su nivel de expresión** entre dos o más condiciones (por ejemplo, una **condición basal** frente a una **condición experimental**), lo que permite identificar posibles **genes regulados diferencialmente** y comprender mejor los procesos biológicos subyacentes.

## 2. Flujo general del análisis de expresión diferencial

De manera general, los pasos a seguir para realizar un **análisis de expresión diferencial** se pueden resumir en las siguientes etapas:

1. **Lectura de datos:** Importar la **tabla de abundancias** o matriz de conteos que contiene el número de lecturas por gen y por muestra.  
2. **Análisis exploratorio de los datos:** Generar **gráficas y estadísticas descriptivas** que permitan obtener una visión general y detectar posibles anomalías o sesgos en los datos.  
3. **Filtrado de datos:** Eliminar genes con **baja expresión** o conteos poco significativos para evitar ruido en el análisis.  
4. **Normalización:** Ajustar los datos para hacer las **muestras comparables** entre sí, compensando diferencias en la profundidad de secuenciación.  
5. **Pruebas estadísticas:** Aplicar métodos estadísticos para **inferir la expresión diferencial**, es decir, detectar los genes cuyo nivel de expresión cambia significativamente entre condiciones más allá de la variación aleatoria.  
6. **Interpretación de resultados:** Clasificar los genes como **no significativos**, **sobreexpresados** o **subexpresados**, de acuerdo con los valores de corte seleccionados para el nivel de significancia (*padjust*, FDR) y el cambio de expresión (*log₂ fold change*).  
7. **Representación visual de los resultados:** Crear gráficas como **MA plots**, **volcano plots** o **heatmaps** para facilitar la interpretación y comunicación de los hallazgos.  
8. **Análisis posterior:** Explorar las **funciones biológicas** de los genes diferencialmente expresados mediante análisis de enriquecimiento o anotación funcional.

En las siguientes secciones realizaremos de manera práctica los pasos del **1 al 7**, utilizando **Python** y la biblioteca **pyDESeq**.

## 3. Lectura de datos (tabla de conteos)

En este ejemplo trabajaremos con la **tabla de conteos** relacionada con la expresión del gen **Pasilla** en *Drosophila melanogaster*, generada previamente en la clase de alineamiento. Esta tabla contiene el **número de lecturas asignadas a cada gen** en las **siete muestras** asociadas a las condiciones *treated* (3 muestras) y *untreated* (4 muestras). Estos datos constituyen el **punto de partida** para el análisis de expresión diferencial.

Para la lectura de los datos utilizaremos la biblioteca [`pandas`](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf), una herramienta fundamental en Python para la manipulación y análisis de datos en formato tabular.


In [ ]:
# Cargar la biblioteca pandas
import pandas as pd

df_count = pd.read_csv('tmp\All_counts_pasilla.txt', sep="\t", index_col=0)
display(df_count)





# Imprimesión del número de renglones y columnas
df_count.shape


Cambiaremos y reordenaremos los nombres de las muestras para que sean más claros. Las muestras **GSM461176–GSM461178** y **GSM461182** pertenecen a la condición *“Untreated”*, mientras que las muestras **GSM461179–GSM461181** corresponden a la condición *“Treated”*. Esta asignación facilitará la interpretación de los resultados durante el análisis diferencial de expresión.

In [ ]:
# Reordenar el DataFrame



# Reasignando los nombres de columnas





## 4. Análisis exploratorio de los datos

Una vez filtrados los datos, realizaremos un análisis exploratorio para examinar su comportamiento y distribución. Este análisis se llevará a cabo de forma visual, mediante una serie de gráficas que nos permitirán identificar patrones o posibles anomalías en los datos.  

Para ello utilizaremos la biblioteca **`seaborn`**, que se estudió en la sesión previa.  


### 4.1 Distribución de los conteos por muestra

La primera gráfica que generaremos será un **boxplot**, en el que el eje **X** representa las muestras y el eje **Y** la **distribución de los conteos**.  

Esta gráfica es importante porque permite **visualizar la variabilidad y la dispersión** de los datos entre muestras, identificar **valores atípicos** y detectar posibles **diferencias en la escala o distribución** de los conteos. En el contexto del análisis de expresión génica, el boxplot ayuda a evaluar si las muestras presentan comportamientos similares o si existen sesgos sistemáticos que podrían afectar el análisis posterior.


> **Nota:**  
> Para todas las gráficas que generaremos en esta sección, trabajaremos con los datos **transformados**, aplicando una transformación **log₂**. Esta transformación actúa como una **pseudo-normalización**, permitiendo manejar mejor la **distribución asimétrica de los conteos** y facilitando la **comparación visual entre muestras**.
>
> Para realizar la transformación utilizaremos la función `log2` de la biblioteca **NumPy**.


In [ ]:
# Cargando la biblioteca
import numpy as np

# Calcular el log2 de la tabla de conteos



Por otra parte, los datos están en **formato ancho (wide)**, pero necesitamos que estén en **formato largo (long)** para trabajar con ellos en `seaborn`. Para ello utilizaremos la función `melt` de **pandas**, que transforma la tabla adecuadamente. Además, agregaremos una columna con el nombre de la **condición** a la que pertenece cada muestra:

In [ ]:
# Transformando la tabla de formato wide a formato long


# Incorporando la columna Cond, para indicar la condición de cada muestra




Con `df_LogMelt`, podemos generar la **gráfica de cajas** utilizando la función `boxplot` de **seaborn**. Esta gráfica permitirá visualizar la **distribución de los conteos** por muestra y condición.

In [ ]:
# Importando las bibliotecas de graficación
import seaborn as sns
import matplotlib.pyplot as plt

# Generando el boxplot
colores_lista = ["skyblue", "forestgreen"]


#Agregando titulos




# Ajustar posición de la leyenda y cambiar tamaño de letra





Las siguientes gráficas que exploraremos son las de **densidad**. Estas gráficas permiten visualizar la **distribución de los valores** de manera continua, mostrando dónde se concentran los datos y facilitando la comparación entre muestras o condiciones.

Para generar estas gráficas utilizaremos la función `displot` de **Seaborn** con el tipo `kde` (Kernel Density Estimation), que permite trazar la **estimación de la densidad**:

In [ ]:
# Generando las gráficas de densidad




#Agregando titulos






### 4.2 PCA

Para explorar la **variabilidad general de los datos** y detectar patrones entre muestras, realizamos un **Análisis de Componentes Principales (PCA)**.

El **PCA** reduce la dimensionalidad de los datos al transformar las variables originales en **componentes principales** ortogonales que capturan la mayor parte de la **varianza total**. Esto permite representar relaciones complejas en un espacio más compacto y fácil de interpretar.

En Python, el cálculo de las **componentes principales** se puede realizar utilizando la clase `PCA` del módulo `sklearn.decomposition`, como se muestra en el siguiente código:


In [ ]:
# Cargando la función PCA de la biblioteca sklearn
from sklearn.decomposition import PCA

# Transponer para que los genes queden en columnas


# Inicia el modelo de Análisis de Componentes Principales, para evaluar 2 componentes




# Calculando las componentes principales




# Creando un dataframe con las componentes, para su posterior graficación





Una vez calculadas las **componentes principales**, podemos **visualizar la dispersión de las muestras** en el espacio formado por las dos primeras componentes (PC1 y PC2), mediante un **diagrama de dispersión**.  

Para ello, utilizamos la función `scatterplot` de **seaborn**, que permite representar cada muestra como un punto y diferenciar las condiciones experimentales mediante colores:


In [ ]:
# Graficando la pca con un scatterplot
formas=["^","D"]




# Ajustar su posición de la leyenda y cambiar tamaño de letra





# Examinando las variables que tiene la variación




El sitio donde se puede obtener la información en Seaborn es https://seaborn.pydata.org/tutorial/properties.html?utm_source=chatgpt.com y en Matplotlib es https://matplotlib.org/stable/api/markers_api.html?utm_source=chatgpt.com. A continuación se presenta una tabla con algunos de ellos.

| Código | Descripción                           |     Código     | Descripción                             |                  |
| :----: | ------------------------------------- | :------------: | --------------------------------------- | ---------------- |
|  `"."` | Punto                                 |      `","`     | Pixel (más pequeño)                     |                  |
|  `"o"` | Círculo                               |      `"v"`     | Triángulo hacia abajo                   |                  |
|  `"^"` | Triángulo hacia arriba                |      `"<"`     | Triángulo hacia la izquierda            |                  |
|  `">"` | Triángulo hacia la derecha            |      `"1"`     | Triángulo hacia abajo (estrecho)        |                  |
|  `"2"` | Triángulo hacia arriba (estrecho)     |      `"3"`     | Triángulo hacia la izquierda (estrecho) |                  |
|  `"4"` | Triángulo hacia la derecha (estrecho) |      `"s"`     | Cuadrado                                |                  |
|  `"p"` | Pentágono                             |      `"P"`     | Cruz gruesa (plus filled)               |                  |
|  `"*"` | Estrella                              |      `"h"`     | Hexágono 1                              |                  |
|  `"H"` | Hexágono 2                            |      `"+"`     | Cruz                                    |                  |
|  `"x"` | X                                     |      `"X"`     | X gruesa (filled)                       |                  |
|  `"D"` | Diamante                              |      `"d"`     | Diamante delgado                        |                  |
|  `"\|"` | Línea vertical                              |      `"_"`     | Línea horizontal                        |                  |


## 5. Filtrando datos poco significativos

Una manera sencilla de filtrar los datos es eliminar aquellos genes que tengan cero conteos en todas las muestras. Esto permite reducir el tamaño de la tabla y enfocar el análisis únicamente en los genes que presentan algún nivel de expresión, como se muestra en el siguiente código:

In [ ]:
# Filtrando los genes que tienen al menos un conteo mayor a cero en alguna columna



# Impresión del número de filas resultantes




Para un análisis más robusto, se utilizan los **conteos por millón (CPM)**, que ajustan los conteos de cada gen según la **profundidad de secuenciación** de cada muestra. Así, se calcula la proporción de lecturas por gen, se multiplica por 1 millón y se filtran los genes con expresión muy baja en la mayoría de las muestras, siguiendo lo recomendado por **Anders et al. (2013)**, quienes sugieren filtrar genes con CPM menor a un umbral **U** en al menos **n muestras**, donde **n** es el tamaño del grupo más pequeño de réplicas.  

A continuación definiremos la función `cpm()` y filtraremos los genes con **CPM > 5 en al menos 3 muestras** (ya que el grupo más pequeño de réplicas es 3), eliminando los genes poco expresados:

In [ ]:
# Función para calcular los conteos por millón

def cpm(df):
    """
    Calcula los conteos por millón (CPM) de un DataFrame de conteos brutos.

    Args:
        df (pd.DataFrame): DataFrame con valores numéricos (conteos de genes)
    
    Returns:
        pd.DataFrame: DataFrame con los valores de CPM
    """
    return (df / df.sum()) * 1000000

In [ ]:
# Filtrando la tabla de conteos, con U = 5 
# en un número mínimo de muestras n= 3




# Impresión del número de filas resultantes



# Guardando el dataframe con la matriz de conteos filtrada





## 6. Análisis de expresión diferencial  

Para realizar el **análisis de expresión diferencial**, utilizaremos la biblioteca **pydeseq2**.

Como paso inicial, reorganizaremos la **tabla de conteos** para que los **genes estén en columnas** y las **muestras en filas**, el formato requerido por esta biblioteca.


In [ ]:
# Cargando los modulos de pydeseq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats
from pydeseq2.preprocessing import deseq2_norm

import os

# Haciendo la transpuesta de la tabla de conteos
df_CountFilter_T = df_CountFilter.T
display(df_CountFilter_T.head())

### 6.1 Creación de la matriz de diseño o metadata  

Primero, creamos la **matriz de diseño** (o **metadata**), que especifica la **asociación entre las muestras y sus respectivas condiciones experimentales**. Esta información es fundamental para que PyDESeq2 pueda identificar qué grupos serán comparados durante el análisis.



In [ ]:
# Creación de la matriz de diseño
metadata = pd.DataFrame(Condiciones, columns=["condition"], index=df_CountFilter_T.index)
print(metadata)

### 6.2 Creación del objeto DESeqData

DESeq2 trabaja con objetos de tipo `DESeqData`, los cuales almacenan toda la información necesaria para realizar el análisis de **expresión diferencial**. 

Para crear este objeto, se requiere **la tabla de conteos**, la **matriz de diseño** y el nombre de la **columna** dentro de la metadataque contiene la **condición experimental de cada muestra**.

La creación del objeto se realiza mediante la función `DeseqDataSet`

In [ ]:
# Creando el objeto dds
dds = DeseqDataSet(
    counts = df_CountFilter_T,
    metadata = metadata
)
print(dds)

### 6.3 Normalización y cálculo de dispersión


Una vez creado el objeto `DeseqDataSet`, se ejecuta el método `deseq2()`, el cual lleva a cabo varios pasos fundamentales del análisis de expresión diferencial.  

Primero, realiza la **normalización** de los conteos mediante la estimación de **factores de tamaño**, que compensan las diferencias en la **profundidad de secuenciación** entre muestras. Después, estima la **dispersión** de cada gen, que representa la **variabilidad biológica** entre réplicas, y finalmente ajusta el modelo estadístico para calcular los **cambios en la expresión génica (log₂ fold change)** entre las condiciones experimentales.  

In [ ]:
# Aplicar el método deseq2 a los datos
dds.deseq2()

En este punto, los **parámetros del análisis** se almacenan organizando los campos por claves. En particular:

- **`X`** almacena los **datos de conteo**.  
- **`obs`** contiene información **a nivel de muestra (1D)**, como los **factores de diseño** y los **`size_factors`** utilizados en la normalización.  
- **`obsm`** guarda información **multidimensional a nivel de muestra**, como la **matriz de diseño (`design_matrix`)**.  
- **`var`** almacena información **a nivel de gen (1D)**, como los **nombres de los genes** y las **dispersiónes (`dispersions`)** estimadas.  
- **`varm`** contiene información **multidimensional a nivel de gen**, como los **valores de cambio en la expresión (`LFC`)**.

Como ejemplo, a continuación se muestra cómo podemos acceder a las  muestras y valores de **normalización**, a las **dispersiónes** y a los **valores de cambio en la expresión (LFCs)** en **escala logarítmica natural**:



In [ ]:
# Solicitando la información de las muestras
print("***** Información de las muestras *****")
print(dds.obs)

# Solicitando la información de la dispersión
print("***** Información de la dispersión *****")
print(dds.varm["dispersions"])

# Solicitando los valores de cambio LFC
print("***** Información de LFC *****")
print(dds.varm["LFC"])

### 6.4 Análisis estadístico con la clase `DeseqStats`

Una vez realizada la **normalización**, el cálculo de la **dispersión** y obtenidos los **valores de cambio en la expresión (LFC)**, se pueden realizar las **pruebas estadísticas** para calcular los **p-values** y los **p-values ajustados**, fundamentales para identificar genes diferencialmente expresados.

Para ello, utilizaremos la función `DeseqStats`, que requiere dos argumentos obligatorios:

- **`dds`**: el objeto `DESeqDataSet` que contiene la información de **normalización** y **dispersión**.  
- **`contrast`**: una lista de 3 cadenas de la forma `["variable", "condición_prueba", "condición_basal"]`, que define la comparación de interés.

La función crea un objeto de la clase `DeseqStats`, que se utilizará posteriormente para calcular los **p-values** y realizar el análisis estadístico de la expresión diferencial.

In [ ]:
# Creando el objeto de tipo DeseqStats
ds = DeseqStats(dds, contrast=["condition", "Treated","Untreated"])
print(ds)

Finalmente, utilizando la función `summary()` se ejecutan todas las **pruebas estadísticas**, calculando los **p-values** y los **p-values ajustados**, haciendo uso de la información de normalización y dispersión previamente almacenada en el objeto `DeseqStats`.

In [ ]:
# EvaluandoCalculando las pruebas estadísticas
ds.summary()

Los resultados obtenidos al aplicar la función `summary()` se almacenan en el atributo `results_df`, que es un **DataFrame**.  

In [ ]:
# Guardando los resultados en res_df_pyDESeq2
res_df_pyDESeq2 = ds.results_df
res_df_pyDESeq2.head()

# Guardando los resultados en un archivo
res_df_pyDESeq2.to_csv("../data/ExpDifRes.txt") 

Entre los atributos disponibles se encuentran:

- **`padj`**: p-value ajustado, útil para controlar la tasa de falsos positivos.  
- **`log2FoldChange`**: cambio en la expresión en escala log₂ entre condiciones.

Estos valores permiten **identificar y filtrar genes con expresión diferencial significativa**.

## 7. Interpretación de resultados

### 7.1 Genes sub y sobre expresados

Para obtener los nombres de los genes que se encuentran **sobreexpresados** o **subexpresados**, realizaremos un **filtrado de los resultados**, considerando los siguientes criterios:

- **Magnitud del cambio (`log2FoldChange`)**: al menos ±2  
- **p-value ajustado (`padj`)**: ≤ 0.01

Este filtrado nos permitirá identificar genes con cambios de expresión **estadísticamente significativos** y de **magnitud relevante**.

Esta tarea la realizaremos con la función `select` de la biblioteca **NumPy**:


In [ ]:
# Si los resultados de la ED, están en un archivo. Leer el archivo


# Definir las condiciones con los valores de corte
cond = [ (res_df_pyDESeq2["padj"] <= 0.01) & (res_df_pyDESeq2["log2FoldChange"] >= 2),(res_df_pyDESeq2["padj"] <= 0.01) & (res_df_pyDESeq2["log2FoldChange"] <= -2)]

# Definir los valores correspondientes
opc = ["UP", "DOWN"]

# Crear la nueva columna


# Desplegando el dataframe





Para conocer cuántos valores hay en cada categoría (**UP**, **DOWN** y **Non-DE**), utilizaremos el método `value_counts()` de **pandas**.


In [ ]:
# Contando los valores asociados a cada categoría de la columna Expresion





### 7.2 Interpretación grafica de los resultados

Una forma eficaz de **resumir e interpretar** los resultados del análisis de expresión diferencial es mediante su **visualización gráfica**. Para ello, utilizaremos dos tipos de representaciones: el **volcano plot** y los **mapas de calor (heatmaps)**, que permiten identificar patrones y genes diferencialmente expresados de manera más intuitiva.

### 7.2.1 Gráfica de Volcano

El **volcano plot** es una gráfica de dispersión que combina la **magnitud del cambio de expresión** (`log2FoldChange`) con la **significancia estadística** (`-log10(padj)`), permitiendo identificar de forma rápida los genes **sobreexpresados** y **subexpresados** de manera significativa.

Antes de generar la gráfica, calcularemos el valor de `-log10(padj)` utilizando la biblioteca **NumPy**, ya que este valor se empleará para representar el eje **Y** de la gráfica.


In [ ]:
# Crear la columna log10Neg




In [ ]:
# Creando la gráfica de puntos
plt.figure(figsize=(4,6))
colores = {
    "UP": "red",
    "DOWN": "forestgreen",
    "Non-DE": "darkgray",
}








### 7.2.2 Heatmap

Además de la **gráfica de Volcano**, también podemos generar un **mapa de calor (heatmap)** para visualizar y contrastar la expresión de los **genes diferencialmente expresados** de manera significativa entre condiciones.  

Para ello, primero **obtenemos los conteos normalizados** utilizando la función `deseq2_norm()` y seleccionamos únicamente los **genes significativos** (aquellos clasificados como *UP* o *DOWN*).  

Posteriormente, aplicamos una **normalización tipo Z-score por gen**, lo que permite centrar y escalar los valores de expresión, facilitando la comparación entre muestras y genes.  

Finalmente, utilizamos la función `clustermap()` de **seaborn**, que genera el mapa de calor agrupando tanto los genes como las muestras según su similitud en los patrones de expresión.


In [ ]:
# Obtener los conteos normalizados, estos dependen de la biblioteca de pydeseq2
# Existen dos maneras de obtenerlos
# 1. Con la función deseq2_norm(
#normalizacion = deseq2_norm(df_CountFilter_T)

# 2. Directamente del objeto dds
#normCounts = pd.DataFrame(dds.layers['normed_counts'].T,index=dds.var_names,columns=dds.obs_names)#normalizacion[0]

#display(normCounts)


In [ ]:
# Si los conteos normalizados están en un archivo. Leer el archivo
normCounts = pd.read_csv("resultados/ConteosNormaliz.csv", index_col = 0)

# Seleccionando los genes significativos


# Aplicando un logaritmo a los conteos para achatar picos





# Graficando el heatmap
import seaborn as sns
import matplotlib.pyplot as plt





